## 从雅虎财经网站获取股票数据

In [18]:
import urllib
import os
import datetime
import pandas as pd


In [10]:
# 下载整个股票数据
def retrive_stock_data(stockid, postfix, folder):
    print('downloading %s.%s to %s' % (stockid, postfix, folder))
    url = 'http://table.finance.yahoo.com/table.csv?s=%s.%s' % (stockid, postfix)
    fname = os.path.join(folder, '%s.csv' % stockid)
    if not os.path.isdir(folder):
        os.mkdir(folder)
    urllib.urlretrieve(url, fname)

In [11]:
retrive_stock_data(600690, 'ss', 'yahoo-data')

downloading 600690.ss to yahoo-data


In [56]:
# 更新股票数据，如果不存在，则下载。如果存在，则只更新最近日期的数据
def update_stock_data(stockid, postfix, folder):
    fname = os.path.join(folder, '%s.csv' % stockid)
    if not os.path.exists(fname):
        retrive_stock_data(stockid, postfix, folder)
        return
    
    data = pd.read_csv(fname, index_col='Date', parse_dates=True)
    
    last_date = data.iloc[0:1].index.tolist()[0]
    today = pd.Timestamp(datetime.date.today())
    if today - last_date < pd.Timedelta(days=1) or \
        (today - last_date == pd.Timedelta(days=1) and datetime.time().hour < 15):
        print('Nothing to update. %s.%s last date is %s.' % (stockid, postfix, last_date))
        return
    
    print('updatting %s.%s to from %s to %s' % (stockid, postfix, last_date, today))
    url = 'http://table.finance.yahoo.com/table.csv?a=%s&b=%s&c=%s&d=%s&e=%s&f=%s&s=%s.%s' \
            % (last_date.month - 1, last_date.day, last_date.year, 
               today.month - 1, today.day, today.year, stockid, postfix)
    temp_file = fname + '.tmp'
    urllib.urlretrieve(url, temp_file)
    update_data = pd.read_csv(temp_file, index_col='Date', parse_dates=True)
    data = data.append(update_data)
    data.to_csv(fname, mode='w')

In [57]:
update_stock_data(600690, 'ss', 'yahoo-data')

updatting 600690.ss to from 2015-11-18 00:00:00 to 2015-11-26 00:00:00
http://table.finance.yahoo.com/table.csv?a=10&b=18&c=2015&d=10&e=26&f=2015&s=600690.ss
yahoo-data/600690.csv.tmp
5
5715
5720


In [38]:
pd.DataFrame.append?